d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Connecting to Azure Blob Storage

Apache Spark&trade; and Databricks&reg; allow you to connect to virtually any data store including Azure Blob Storage.
## In this lesson you:
* Mount and access data in Azure Blob Storage
* Define options when reading from Azure Blob Storage

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Scientists and Data Pipeline Engineers

## Prerequisites
* Web browser: Please use a <a href="https://docs.azuredatabricks.net/user-guide/supported-browsers.html#supported-browsers" target="_blank">supported browser</a>.
* Concept (optional): <a href="https://academy.databricks.com/collections/frontpage/products/dataframes" target="_blank">DataFrames course from Databricks Academy</a>

<iframe  
src="//fast.wistia.net/embed/iframe/8qe9xs3k7u?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/8qe9xs3k7u?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Spark as a Connector

Spark quickly rose to popularity as a replacement for the [Apache Hadoop&trade;](http://hadoop.apache.org/) MapReduce paradigm in a large part because it easily connected to a number of different data sources.  Most important among these data sources was the Hadoop Distributed File System (HDFS).  Now, Spark engineers connect to a wide variety of data sources including:  
<br>
* Traditional databases like Postgres, SQL Server, and MySQL
* Message brokers like <a href="https://kafka.apache.org/" target="_blank">Apache Kafka</a> and <a href="https://docs.microsoft.com/en-us/azure/event-hubs/event-hubs-about">Azure Event Hubs</a>
* Distributed databases like Cassandra and Redshift
* Data warehouses like Hive and Cosmos DB
* File types like CSV, Parquet, and Avro

<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/open-source-ecosystem_2.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

-sandbox
### DBFS Mounts and Azure Blob Storage

Azure Blob Storage is the backbone of Databricks workflows.  Azure Blob Storage offers data storage that easily scales to the demands of most data applications and, by colocating data with Spark clusters, Databricks quickly reads from and writes to Azure Blob Storage in a distributed manner.

The Databricks File System (DBFS), is a layer over Azure Blob Storage that allows you to mount Blob containers, making them available to other users in your workspace and persisting the data after a cluster is shut down.

In our road map for ETL, this is the <b>Extract and Validate </b> step:

<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/ETL-Process-1.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

<iframe  
src="//fast.wistia.net/embed/iframe/sls8z8pw8n?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/sls8z8pw8n?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Run the cell below to set up your environment.

In [8]:
%run "./Includes/Classroom-Setup"

-sandbox

Define your Azure Blob credentials.  You need the following elements:<br><br>

* Storage account name
* Container name
* Mount point (how the mount will appear in DBFS)
* Shared Access Signature (SAS) key

Below these elements are defined, including a read-only SAS key.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> For more information on SAS keys, <a href="https://docs.microsoft.com/en-us/azure/storage/common/storage-dotnet-shared-access-signature-part-1" target="_blank"> see the Azure documentation.</a><br>
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> SAS keys are normally provided as a SAS URI. Of this URI, focus on everything from the `?` on, including the `?`. The following cell provides an example of this.

In [10]:
STORAGE_ACCOUNT = "dbtraineastus2"
CONTAINER = "training"
MOUNT_POINT = "/mnt/training"
SAS_KEY = "?sv=2017-07-29&ss=b&srt=sco&sp=rl&se=2023-04-19T06:32:30Z&st=2018-04-18T22:32:30Z&spr=https&sig=BB%2FQzc0XHAH%2FarDQhKcpu49feb7llv3ZjnfViuI9IWo%3D"

First, unmount the container, since you mounted this container in the classroom setup script.

In [12]:
try:
  dbutils.fs.unmount(MOUNT_POINT) # Use this to unmount as needed
except:
  print("{} already unmounted".format(MOUNT_POINT))

/mnt/training has been unmounted.

Define two strings populated with the storage account and container information.  This will be passed to the `mount` function.

In [14]:
source_str = "wasbs://{container}@{storage_acct}.blob.core.windows.net/".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
conf_key = "fs.azure.sas.{container}.{storage_acct}.blob.core.windows.net".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)

-sandbox

Now, mount the container <a href="https://docs.databricks.com/spark/latest/data-sources/azure/azure-storage.html#mount-azure-blob-storage-containers-with-dbfs" target="_blank"> using the template provided in the docs.</a>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The code below includes error handling logic to handle the case where the mount is already mounted.

In [16]:
try:
  dbutils.fs.mount(
    source = source_str,
    mount_point = MOUNT_POINT,
    extra_configs = {conf_key: SAS_KEY}
  )
except Exception as e:
  print("ERROR: {} already mounted. Run previous cells to unmount first".format(MOUNT_POINT))

Next, explore the mount using `%fs ls` and the name of the mount.

In [18]:
%fs ls /mnt/training

path,name,size
dbfs:/mnt/training/301/,301/,0
dbfs:/mnt/training/Chicago-Crimes-2018.csv,Chicago-Crimes-2018.csv,5201668
dbfs:/mnt/training/City-Data.parquet/,City-Data.parquet/,0
dbfs:/mnt/training/EDGAR-Log-20170329/,EDGAR-Log-20170329/,0
dbfs:/mnt/training/StatLib/,StatLib/,0
dbfs:/mnt/training/UbiqLog4UCI/,UbiqLog4UCI/,0
dbfs:/mnt/training/_META/,_META/,0
dbfs:/mnt/training/adventure-works/,adventure-works/,0
dbfs:/mnt/training/airbnb/,airbnb/,0
dbfs:/mnt/training/airbnb-sf-listings.csv,airbnb-sf-listings.csv,764121


-sandbox

In practice, always secure your credentials.  Do this by either maintaining a single notebook with restricted permissions that holds SAS keys, or delete the cells or notebooks that expose the keys. **After a cell used to mount a container is run, access this mount in any notebook, any cluster, and share the mount between colleagues.**

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://docs.azuredatabricks.net/user-guide/secrets/index.html" target="_blank">secret management to securely store and reference your credentials in notebooks and jobs.</a>

## Adding Options

When you import that data into a cluster, you can add options based on the specific characteristics of the data.

<iframe  
src="//fast.wistia.net/embed/iframe/6pckay2lii?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/6pckay2lii?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Display the first few lines of `Chicago-Crimes-2018.csv` using `%fs head`.

In [23]:
%fs head /mnt/training/Chicago-Crimes-2018.csv

[Truncated to first 65536 bytes]
ID	Case Number	Date	Block	IUCR	Primary Type	Description	Location Description	Arrest	Domestic	Beat	District	Ward	Community Area	FBI Code	X Coordinate	Y Coordinate	Year	Updated On	Latitude	Longitude	Location
23811	JB141441	02/05/2018 01:10:00 AM	118XX S INDIANA AVE	0110	HOMICIDE	FIRST DEGREE MURDER	VACANT LOT	false	false	0532	005	9	53	01A	1179707	1826280	2018	02/12/2018 03:49:14 PM	41.678585145	-87.617837834	(41.678585145, -87.617837834)
11228589	JB148990	01/23/2018 09:00:00 AM	072XX S VERNON AVE	1153	DECEPTIVE PRACTICE	FINANCIAL IDENTITY THEFT OVER $ 300	OTHER	false	false	0323	003	6	69	11 2018	02/12/2018 03:49:14 PM 
11228563	JB148931	01/31/2018 10:12:00 AM	040XX N KEYSTONE AVE	1154	DECEPTIVE PRACTICE	FINANCIAL IDENTITY THEFT $300 AND UNDER	APARTMENT	false	false	1722	017	39	16	11 2018	02/12/2018 03:49:14 PM 
11228555	JB148885	02/01/2018 02:00:00 PM	017XX W CONGRESS PKWY	0820	THEFT	$500 AND UNDER	HOSPITAL BUILDING/GROUNDS	false	false	1231	012	2	28	06 2018	02/12/2018 03:49:14 PM 
11228430	JB148675	01/27/2018 09:00:00 PM	061XX S EBERHART AVE	0560	ASSAULT	SIMPLE	RESIDENCE	false	true	0313	003	20	42	08A 2018	02/12/2018 03:49:14 PM 
11228401	JB148683	02/02/2018 12:00:00 PM	038XX N SAWYER AVE	1153	DECEPTIVE PRACTICE	FINANCIAL IDENTITY THEFT OVER $ 300	RESIDENCE	false	false	1733	017	33	16	11 2018	02/12/2018 03:49:14 PM 
11228347	JB148599	01/28/2018 07:00:00 PM	008XX E 45TH ST	0620	BURGLARY	UNLAWFUL ENTRY	RESIDENCE	false	false	0221	002	4	39	05 2018	02/12/2018 03:49:14 PM 
11228291	JB148591	01/10/2018 04:45:00 PM	010XX E 53RD ST	1153	DECEPTIVE PRACTICE	FINANCIAL IDENTITY THEFT OVER $ 300 false	false	0233	002	4	41	11 2018	02/12/2018 03:49:14 PM 
11228287	JB148482	01/03/2018 03:45:00 PM	0000X W C1 ST	0810	THEFT	OVER $500	AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA	false	false	1651	016	41	76	06 2018	02/12/2018 03:49:14 PM 
11228268	JB148558	02/04/2018 04:00:00 PM	044XX S MICHIGAN AVE	2825	OTHER OFFENSE	HARASSMENT BY TELEPHONE	APARTMENT	false	true	0215	002	3	38	26 2018	02/12/2018 03:49:14 PM 
11228252	JB148398	01/27/2018 07:00:00 AM	009XX N LONG AVE	1150	DECEPTIVE PRACTICE	CREDIT CARD FRAUD	APARTMENT	false	false	1524	015	37	25	11 2018	02/12/2018 03:49:14 PM 
11228249	JB148428	02/05/2018 12:00:00 AM	095XX S NORMAL AVE	4387	OTHER OFFENSE	VIOLATE ORDER OF PROTECTION	RESIDENCE	false	true	2223	022	21	73	26 2018	02/12/2018 03:49:14 PM 
11228245	JB148296	01/21/2018 02:00:00 AM	044XX W MONTROSE AVE	0486	BATTERY	DOMESTIC BATTERY SIMPLE	APARTMENT	false	true	1722	017	38	16	08B 2018	02/12/2018 03:49:14 PM 
11228216	JB138205	02/02/2018 12:45:00 AM	008XX W 76TH ST	0486	BATTERY	DOMESTIC BATTERY SIMPLE	APARTMENT	false	true	0621	006	17	71	08B 2018	02/12/2018 03:49:14 PM 
11228185	JB147437	01/25/2018 01:14:00 PM	120XX S HALSTED ST	4651	OTHER OFFENSE	SEX OFFENDER: FAIL REG NEW ADD	APARTMENT	false	false	0523	005	34	53	26 2018	02/12/2018 03:49:14 PM 
11228151	JB148383	02/05/2018 05:00:00 PM	029XX W 82ND ST	1220	DECEPTIVE PRACTICE	THEFT OF LOST/MISLAID PROP	OTHER	false	false	0835	008	18	70	11 2018	02/12/2018 03:49:14 PM 
11228147	JB148407	01/15/2018 12:00:00 AM	025XX N KEDZIE BLVD	1153	DECEPTIVE PRACTICE	FINANCIAL IDENTITY THEFT OVER $ 300	RESIDENCE	false	false	1413	014	35	22	11 2018	02/12/2018 03:49:14 PM 
11228087	JB143362	02/05/2018 07:30:00 PM	019XX N KILDARE AVE	1750	OFFENSE INVOLVING CHILDREN	CHILD ABUSE	RESIDENCE	false	true	2534	025	30	20	20 2018	02/12/2018 03:49:14 PM 
11227938	JB140860	02/02/2018 08:30:00 PM	018XX S ST LOUIS AVE	1320	CRIMINAL DAMAGE	TO VEHICLE	STREET	false	false	1021	010	24	29	14 2018	02/12/2018 03:49:14 PM 
11227926	JB148107	02/03/2018 06:00:00 PM	073XX S ST LAWRENCE AVE	2825	OTHER OFFENSE	HARASSMENT BY TELEPHONE	RESIDENCE	false	false	0323	003	6	69	26 2018	02/12/2018 03:49:14 PM 
11227921	JB148091	01/05/2018 04:00:00 PM	043XX W ADAMS ST	5002	OTHER OFFENSE	OTHER VEHICLE OFFENSE	STREET	false	true	1115	011	28	26	26 2018	02/12/2018 03:49:14 PM 
11227906	JB148074	01/28/2018 12:00:00 PM	037XX S LANGLEY AVE	2

`option` is a method of `DataFrameReader`. Options are key/value pairs and must be specified before calling `.csv()`.

This is a tab-delimited file, as seen in the previous cell. Specify the `"delimiter"` option in the import statement.  

:NOTE: Find a [full list of parameters here.](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dateformat#pyspark.sql.DataFrameReader.csv)

In [25]:
display(spark.read
  .option("delimiter", "\t")
  .csv("/mnt/training/Chicago-Crimes-2018.csv")
)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21
ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
23811,JB141441,02/05/2018 01:10:00 AM,118XX S INDIANA AVE,0110,HOMICIDE,FIRST DEGREE MURDER,VACANT LOT,false,false,0532,005,9,53,01A,1179707,1826280,2018,02/12/2018 03:49:14 PM,41.678585145,-87.617837834,"(41.678585145, -87.617837834)"
11228589,JB148990,01/23/2018 09:00:00 AM,072XX S VERNON AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,false,false,0323,003,6,69,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228563,JB148931,01/31/2018 10:12:00 AM,040XX N KEYSTONE AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,false,false,1722,017,39,16,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228555,JB148885,02/01/2018 02:00:00 PM,017XX W CONGRESS PKWY,0820,THEFT,$500 AND UNDER,HOSPITAL BUILDING/GROUNDS,false,false,1231,012,2,28,06,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228430,JB148675,01/27/2018 09:00:00 PM,061XX S EBERHART AVE,0560,ASSAULT,SIMPLE,RESIDENCE,false,true,0313,003,20,42,08A,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228401,JB148683,02/02/2018 12:00:00 PM,038XX N SAWYER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,false,1733,017,33,16,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228347,JB148599,01/28/2018 07:00:00 PM,008XX E 45TH ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,false,false,0221,002,4,39,05,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228291,JB148591,01/10/2018 04:45:00 PM,010XX E 53RD ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,null,false,false,0233,002,4,41,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228287,JB148482,01/03/2018 03:45:00 PM,0000X W C1 ST,0810,THEFT,OVER $500,AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,false,false,1651,016,41,76,06,null,null,2018,02/12/2018 03:49:14 PM,null,null,null


Spark doesn't read the header by default, as demonstrated by the column names of `_c0`, `_c1`, etc. Notice the column names are present in the first row of the DataFrame. 

Fix this by setting the `"header"` option to `True`.

In [27]:
display(spark.read
  .option("delimiter", "\t")
  .option("header", True)
  .csv("/mnt/training/Chicago-Crimes-2018.csv")
)

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
23811,JB141441,02/05/2018 01:10:00 AM,118XX S INDIANA AVE,0110,HOMICIDE,FIRST DEGREE MURDER,VACANT LOT,false,false,0532,005,9,53,01A,1179707,1826280,2018,02/12/2018 03:49:14 PM,41.678585145,-87.617837834,"(41.678585145, -87.617837834)"
11228589,JB148990,01/23/2018 09:00:00 AM,072XX S VERNON AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,false,false,0323,003,6,69,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228563,JB148931,01/31/2018 10:12:00 AM,040XX N KEYSTONE AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,false,false,1722,017,39,16,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228555,JB148885,02/01/2018 02:00:00 PM,017XX W CONGRESS PKWY,0820,THEFT,$500 AND UNDER,HOSPITAL BUILDING/GROUNDS,false,false,1231,012,2,28,06,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228430,JB148675,01/27/2018 09:00:00 PM,061XX S EBERHART AVE,0560,ASSAULT,SIMPLE,RESIDENCE,false,true,0313,003,20,42,08A,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228401,JB148683,02/02/2018 12:00:00 PM,038XX N SAWYER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,false,1733,017,33,16,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228347,JB148599,01/28/2018 07:00:00 PM,008XX E 45TH ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,false,false,0221,002,4,39,05,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228291,JB148591,01/10/2018 04:45:00 PM,010XX E 53RD ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,null,false,false,0233,002,4,41,11,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228287,JB148482,01/03/2018 03:45:00 PM,0000X W C1 ST,0810,THEFT,OVER $500,AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,false,false,1651,016,41,76,06,null,null,2018,02/12/2018 03:49:14 PM,null,null,null
11228268,JB148558,02/04/2018 04:00:00 PM,044XX S MICHIGAN AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,false,true,0215,002,3,38,26,null,null,2018,02/12/2018 03:49:14 PM,null,null,null


Spark didn't infer the schema, or read the timestamp format, since this file uses an atypical timestamp.  Change that by adding the option `"timestampFormat"` and pass it the format used in this file.  

Set `"inferSchema"` to `True`, which triggers Spark to make an extra pass over the data to infer the schema.

In [29]:
crimeDF = (spark.read
  .option("delimiter", "\t")
  .option("header", True)
  .option("timestampFormat", "mm/dd/yyyy hh:mm:ss a")
  .option("inferSchema", True)
  .csv("/mnt/training/Chicago-Crimes-2018.csv")
)
display(crimeDF)

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
23811,JB141441,2018-01-05T01:10:00.000+0000,118XX S INDIANA AVE,0110,HOMICIDE,FIRST DEGREE MURDER,VACANT LOT,false,false,532,5,9,53,01A,1179707,1826280,2018,2018-01-12T15:49:14.000+0000,41.678585145,-87.617837834,"(41.678585145, -87.617837834)"
11228589,JB148990,2018-01-23T09:00:00.000+0000,072XX S VERNON AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,false,false,323,3,6,69,11,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228563,JB148931,2018-01-31T10:12:00.000+0000,040XX N KEYSTONE AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,false,false,1722,17,39,16,11,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228555,JB148885,2018-01-01T14:00:00.000+0000,017XX W CONGRESS PKWY,0820,THEFT,$500 AND UNDER,HOSPITAL BUILDING/GROUNDS,false,false,1231,12,2,28,06,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228430,JB148675,2018-01-27T21:00:00.000+0000,061XX S EBERHART AVE,0560,ASSAULT,SIMPLE,RESIDENCE,false,true,313,3,20,42,08A,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228401,JB148683,2018-01-02T12:00:00.000+0000,038XX N SAWYER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,false,1733,17,33,16,11,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228347,JB148599,2018-01-28T19:00:00.000+0000,008XX E 45TH ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,false,false,221,2,4,39,05,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228291,JB148591,2018-01-10T16:45:00.000+0000,010XX E 53RD ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,null,false,false,233,2,4,41,11,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228287,JB148482,2018-01-03T15:45:00.000+0000,0000X W C1 ST,0810,THEFT,OVER $500,AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,false,false,1651,16,41,76,06,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null
11228268,JB148558,2018-01-04T16:00:00.000+0000,044XX S MICHIGAN AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,false,true,215,2,3,38,26,null,null,2018,2018-01-12T15:49:14.000+0000,null,null,null


## The Design Pattern

Other connections work in much the same way, whether your data sits in Cassandra, Cosmos DB, Redshift, or another common data store.  The general pattern is always:  
<br>
1. Define the connection point
2. Define connection parameters such as access credentials
3. Add necessary options

After adhering to this, read data using `spark.read.options(<option key>, <option value>).<connection_type>(<endpoint>)`.

## Exercise 1: Read Wikipedia Data

Read Wikipedia data from Azure Blob Storage, accounting for its delimiter and header.

### Step 1: Get a Sense for the Data

Take a look at the head of the data, located at `/mnt/training/wikipedia/pageviews/pageviews_by_second.tsv`.

In [33]:
%fs head /mnt/training/wikipedia/pageviews/pageviews_by_second.tsv

[Truncated to first 65536 bytes]
"timestamp"	"site"	"requests"
"2015-03-16T00:09:55"	"mobile"	1595
"2015-03-16T00:10:39"	"mobile"	1544
"2015-03-16T00:19:39"	"desktop"	2460
"2015-03-16T00:38:11"	"desktop"	2237
"2015-03-16T00:42:40"	"mobile"	1656
"2015-03-16T00:52:24"	"desktop"	2452
"2015-03-16T00:54:16"	"mobile"	1654
"2015-03-16T01:18:11"	"mobile"	1720
"2015-03-16T01:30:32"	"desktop"	2288
"2015-03-16T01:32:24"	"mobile"	1609
"2015-03-16T01:42:08"	"desktop"	2341
"2015-03-16T01:45:53"	"mobile"	1704
"2015-03-16T01:55:37"	"desktop"	2554
"2015-03-16T01:57:29"	"mobile"	1825
"2015-03-16T02:03:16"	"desktop"	2492
"2015-03-16T02:10:32"	"mobile"	1667
"2015-03-16T02:16:45"	"desktop"	2452
"2015-03-16T02:19:32"	"desktop"	2412
"2015-03-16T02:20:16"	"desktop"	2350
"2015-03-16T02:22:08"	"mobile"	1802
"2015-03-16T02:33:45"	"desktop"	2462
"2015-03-16T02:35:37"	"mobile"	1580
"2015-03-16T03:00:16"	"mobile"	1703
"2015-03-16T03:06:29"	"desktop"	2476
"2015-03-16T03:13:45"	"mobile"	1548
"2015-03-16T03:19:58"	"desktop"	2265
"2015-03-16T03:25:01"	"desktop"	2341
"2015-03-16T03:38:30"	"desktop"	2198
"2015-03-16T03:52:43"	"desktop"	2283
"2015-03-16T03:57:22"	"mobile"	1410
"2015-03-16T04:03:29"	"mobile"	1360
"2015-03-16T04:16:58"	"mobile"	1383
"2015-03-16T04:22:01"	"mobile"	1413
"2015-03-16T04:28:14"	"desktop"	2374
"2015-03-16T04:30:51"	"desktop"	2177
"2015-03-16T04:35:30"	"mobile"	1265
"2015-03-16T04:42:27"	"desktop"	2152
"2015-03-16T04:47:06"	"mobile"	1391
"2015-03-16T04:55:56"	"desktop"	2211
"2015-03-16T05:06:22"	"desktop"	2105
"2015-03-16T05:08:14"	"mobile"	1218
"2015-03-16T05:10:51"	"mobile"	1208
"2015-03-16T05:19:51"	"desktop"	2180
"2015-03-16T05:20:35"	"desktop"	2238
"2015-03-16T05:22:27"	"mobile"	1206
"2015-03-16T05:35:56"	"mobile"	1134
"2015-03-16T06:00:35"	"mobile"	1050
"2015-03-16T06:09:35"	"desktop"	2224
"2015-03-16T06:10:19"	"desktop"	2295
"2015-03-16T06:23:48"	"desktop"	2020
"2015-03-16T06:42:20"	"desktop"	2027
"2015-03-16T06:44:12"	"mobile"	976
"2015-03-16T06:57:41"	"mobile"	894
"2015-03-16T07:01:56"	"desktop"	2211
"2015-03-16T07:03:48"	"mobile"	916
"2015-03-16T07:22:20"	"mobile"	977
"2015-03-16T07:32:04"	"desktop"	2515
"2015-03-16T07:45:33"	"desktop"	2155
"2015-03-16T07:47:25"	"mobile"	878
"2015-03-16T07:57:09"	"desktop"	2286
"2015-03-16T08:06:41"	"desktop"	2227
"2015-03-16T08:12:04"	"mobile"	948
"2015-03-16T08:18:17"	"desktop"	2113
"2015-03-16T08:20:54"	"desktop"	2227
"2015-03-16T08:25:33"	"mobile"	881
"2015-03-16T08:37:09"	"mobile"	876
"2015-03-16T08:45:59"	"desktop"	2255
"2015-03-16T08:51:02"	"desktop"	2339
"2015-03-16T09:03:41"	"mobile"	839
"2015-03-16T09:09:54"	"desktop"	2389
"2015-03-16T09:10:38"	"desktop"	2411
"2015-03-16T09:15:17"	"mobile"	866
"2015-03-16T09:28:46"	"mobile"	867
"2015-03-16T09:42:59"	"mobile"	863
"2015-03-16T09:54:15"	"desktop"	2635
"2015-03-16T10:02:50"	"mobile"	863
"2015-03-16T10:14:26"	"mobile"	974
"2015-03-16T10:27:55"	"mobile"	863
"2015-03-16T10:39:11"	"desktop"	2391
"2015-03-16T10:53:24"	"desktop"	2236
"2015-03-16T10:58:03"	"mobile"	1088
"2015-03-16T11:19:11"	"mobile"	1039
"2015-03-16T11:31:32"	"desktop"	2538
"2015-03-16T11:33:24"	"mobile"	981
"2015-03-16T11:43:08"	"desktop"	2619
"2015-03-16T11:46:53"	"mobile"	961
"2015-03-16T11:56:37"	"desktop"	2647
"2015-03-16T11:58:29"	"mobile"	1042
"2015-03-16T12:07:03"	"desktop"	2588
"2015-03-16T12:11:32"	"mobile"	988
"2015-03-16T12:21:16"	"desktop"	2663
"2015-03-16T12:23:08"	"mobile"	1141
"2015-03-16T12:34:45"	"desktop"	2576
"2015-03-16T12:36:37"	"mobile"	1037
"2015-03-16T13:01:16"	"mobile"	1124
"2015-03-16T13:07:29"	"desktop"	2871
"2015-03-16T13:14:45"	"mobile"	1071
"2015-03-16T13:26:01"	"desktop"	3211
"2015-03-16T13:39:30"	"desktop"	2943
"2015-03-16T13:43:01"	"desktop"	3200
"2015-03-16T13:56:30"	"desktop"	3027
"2015-03-16T13:58:22"	"mobile"	1236
"2015-03-16T14:04:29"	"mobile"	1249
"2015-03-16T14:17:58"	"mobile"	1282
"2015-03-16T14:23:01"	"mobile"	1341
"2015-03-16T14:29:14"	"desktop"	3283
"2015-03-16T14:31:51"	"desktop"	3202
"2015-03-16T14:36:30"	"mobile"	1358
"2015-03-16T1

### Step 2: Import the Raw Data

Import the data **without any options** and save it to `wikiDF`. Display the result.

In [35]:
# TODO
wikiDF = spark.read.csv("/mnt/training/wikipedia/pageviews/pageviews_by_second.tsv")

In [36]:
# TEST - Run this cell to test your solution

dbTest("ET1-P-03-01-01", 7200001, wikiDF.count())
dbTest("ET1-P-03-01-02", '_c0', wikiDF.columns[0])

print("Tests passed!")

Tests passed!

### Step 3: Import the Data with Options

Import the data with options and save it to `wikiWithOptionsDF`.  Display the result.  Your import statement should account for:<br><br>  

 - The header
 - The delimiter

In [38]:
# TODO
wikiWithOptionsDF = (spark.read
                    .option("delimiter", "\t")
                    .option("header",True)
                    .csv("/mnt/training/wikipedia/pageviews/pageviews_by_second.tsv"))

In [39]:
# TEST - Run this cell to test your solution
cols = wikiWithOptionsDF.columns

dbTest("ET1-P-03-02-01", 7200000, wikiWithOptionsDF.count())

dbTest("ET1-P-03-02-02", True, "timestamp" in cols)
dbTest("ET1-P-03-02-03", True, "site" in cols)
dbTest("ET1-P-03-02-04", True, "requests" in cols)

print("Tests passed!")

Tests passed!

## Review

**Question:** What accounts for Spark's quick rise in popularity as an ETL tool?  
**Answer:** Spark easily accesses data virtually anywhere it lives, and the scalable framework lowers the difficulties in building connectors to access data.  Spark offers a unified API for connecting to data making reads from a CSV file, JSON data, or a database, to provide a few examples, nearly identical.  This allows developers to focus on writing their code rather than writing connectors.

**Question:** What is DBFS and why is it important?  
**Answer:** The Databricks File System (DBFS) allows access to scalable, fast, and distributed storage backed by S3 or the Azure Blob Store.

**Question:** How do you connect your Spark cluster to the Azure Blob?  
**Answer:** By mounting it. Mounts require Azure credentials such as SAS keys and give access to a virtually infinite store for your data. One other option is to define your keys in a single notebook that only you have permission to access. Click the arrow next to a notebook in the Workspace tab to define access permissions.

**Question:** How do you specify parameters when reading data?  
**Answer:** Using `.option()` during your read allows you to pass key/value pairs specifying aspects of your read.  For instance, options for reading CSV data include `header`, `delimiter`, and `inferSchema`.

**Question:** What is the general design pattern for connecting to your data?  
**Answer:** The general design pattern is as follows:
0. Define the connection point
0. Define connection parameters such as access credentials
0. Add necessary options such as for headers or parallelization

## Next Steps

Start the next lesson, [Connecting to JDBC]($./04-Connecting-to-JDBC ).

## Additional Topics & Resources

**Q:** Where can I find more information on DBFS?  
**A:** <a href="https://docs.azuredatabricks.net/user-guide/dbfs-databricks-file-system.html#dbfs" target="_blank">Take a look at the Databricks documentation for more details

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>